## Limpieza de base de datos de vacunas
Cargando los datos



In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np

df_original = pd.read_csv('https://cloud.minsa.gob.pe/s/ZgXoXqK2KLjRLxD/download',sep=",")
df_original.head()


,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210306,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210306,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
2,20210306,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210306,746ad64c513b5d53b987ed48a6b45f77,PERSONAL DE SALUD,54.0,FEMENINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210306,80f0a41dd63af3b96d4a80bfab2eaa61,PERSONAL DE SALUD,35.0,FEMENINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO


Dropeando y ordenado

In [2]:
# Generando una base a parte
df = df_original
# Dando formato de fecha a la fecha de resultado de la prueba del covid
import datetime
df["FECHA_VACUNACION"] = df["FECHA_VACUNACION"].astype(int)
df["FECHA_VACUNACION"] = pd.to_datetime(df['FECHA_VACUNACION'], format='%Y%m%d')
# Drop a las observaciones sin fecha de resultado
df.dropna(subset=['FECHA_VACUNACION'], axis=0, inplace=True)
# Drop a las variables poco relevantes por el momento
df = df.drop(['FECHA_CORTE','GRUPO_RIESGO','EDAD','FABRICANTE','SEXO','DOSIS'], axis=1)
# Reordenamiento de variables
df = df[['DEPARTAMENTO', 'PROVINCIA','DISTRITO','FECHA_VACUNACION','UUID']]
# Ordenando por Dep, Prov, Dist y Fecha
df = df.sort_values(by=['DEPARTAMENTO', 'PROVINCIA','DISTRITO','FECHA_VACUNACION'])
# Agrupando las cantidad de UUID = Personas contagiadas
df = df.groupby(by=['DEPARTAMENTO', 'PROVINCIA','DISTRITO','FECHA_VACUNACION'], as_index=False).count()
# Visualización del resultado
df

,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA_VACUNACION,UUID
0,AMAZONAS,BAGUA,ARAMANGO,2021-02-24,69
1,AMAZONAS,BAGUA,ARAMANGO,2021-02-27,1
2,AMAZONAS,BAGUA,ARAMANGO,2021-03-01,1
3,AMAZONAS,BAGUA,ARAMANGO,2021-03-04,4
4,AMAZONAS,BAGUA,BAGUA,2021-02-10,30
...,...,...,...,...,...
7582,UCAYALI,PADRE ABAD,PADRE ABAD,2021-02-15,41
7583,UCAYALI,PADRE ABAD,PADRE ABAD,2021-02-23,47
7584,UCAYALI,PADRE ABAD,PADRE ABAD,2021-02-24,20
7585,UCAYALI,PADRE ABAD,PADRE ABAD,2021-02-25,4


Formateando fecha para trabajar semanalmente


In [3]:
# Dando el formato de semana
df['SEMANA'] = df["FECHA_VACUNACION"].dt.strftime('%Y-%U')
# Drop fecha larga
df = df.drop(["FECHA_VACUNACION"], axis=1)
# Agrupando por Dep, Prov, Dist, Semana
df = df.groupby(by=['DEPARTAMENTO', 'PROVINCIA','DISTRITO','SEMANA'], as_index=False).sum()
# Visualizando Base
df

,DEPARTAMENTO,PROVINCIA,DISTRITO,SEMANA,UUID
0,AMAZONAS,BAGUA,ARAMANGO,2021-08,70
1,AMAZONAS,BAGUA,ARAMANGO,2021-09,5
2,AMAZONAS,BAGUA,BAGUA,2021-06,251
3,AMAZONAS,BAGUA,BAGUA,2021-07,65
4,AMAZONAS,BAGUA,BAGUA,2021-08,422
...,...,...,...,...,...
3315,UCAYALI,PADRE ABAD,NESHUYA,2021-08,58
3316,UCAYALI,PADRE ABAD,PADRE ABAD,2021-06,86
3317,UCAYALI,PADRE ABAD,PADRE ABAD,2021-07,41
3318,UCAYALI,PADRE ABAD,PADRE ABAD,2021-08,71


Cargando Ubigeos

In [4]:
# Cargando los datos
url = "https://raw.githubusercontent.com/vacunatepe/automatcoviddata/main/ubigeo.csv" 
ubi = pd.read_csv(url,dtype={'Ubigeo': str})

# Ordenando
ubi = ubi[['Departamento','Provincia','Distrito','Ubigeo']]
# Renombrando para combinar
ubi = ubi.rename(columns={'Departamento': 'DEPARTAMENTO','Provincia':'PROVINCIA','Distrito':'DISTRITO','Ubigeo':'UBIGEO'})
# Vistazo
ubi

,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO
0,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,010101
1,AMAZONAS,CHACHAPOYAS,ASUNCION,010102
2,AMAZONAS,CHACHAPOYAS,BALSAS,010103
3,AMAZONAS,CHACHAPOYAS,CHETO,010104
4,AMAZONAS,CHACHAPOYAS,CHILIQUIN,010105
...,...,...,...,...
1869,UCAYALI,PADRE ABAD,IRAZOLA,250302
1870,UCAYALI,PADRE ABAD,CURIMANA,250303
1871,UCAYALI,PADRE ABAD,NESHUYA,250304
1872,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,250305


Asignado ubigeo a la base principal

In [5]:
# Combinando
new_df = pd.merge(df, ubi,  how='left', left_on=['DEPARTAMENTO', 'PROVINCIA','DISTRITO'], right_on = ['DEPARTAMENTO', 'PROVINCIA','DISTRITO'])
# Ordenando
new_df = new_df[['DEPARTAMENTO', 'PROVINCIA','DISTRITO','UBIGEO','SEMANA','UUID']]
# Vistazo
new_df

,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,SEMANA,UUID
0,AMAZONAS,BAGUA,ARAMANGO,010202,2021-08,70
1,AMAZONAS,BAGUA,ARAMANGO,010202,2021-09,5
2,AMAZONAS,BAGUA,BAGUA,010201,2021-06,251
3,AMAZONAS,BAGUA,BAGUA,010201,2021-07,65
4,AMAZONAS,BAGUA,BAGUA,010201,2021-08,422
...,...,...,...,...,...,...
3315,UCAYALI,PADRE ABAD,NESHUYA,250304,2021-08,58
3316,UCAYALI,PADRE ABAD,PADRE ABAD,250301,2021-06,86
3317,UCAYALI,PADRE ABAD,PADRE ABAD,250301,2021-07,41
3318,UCAYALI,PADRE ABAD,PADRE ABAD,250301,2021-08,71


Guardando CSV

In [6]:
new_df.to_csv('vacunaUS.csv')